<a href="https://colab.research.google.com/github/minhvu0510/AIO-Exercise/blob/main/Module3_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.
(a) Build tree với điều kiện sau λ = 0, depth = 1, lr = 0.3

In [ ]:
import numpy as np

X = np.array([23, 24, 26, 27])
Y = np.array([50, 70, 80, 85])

y_pred_initial = np.mean(Y)
print(f"Giá trị dự đoán ban đầu: {y_pred_initial}")

residuals = Y - y_pred_initial
print(f"Phần dư: {residuals}")

left_indices = X <= 24
right_indices = X > 24

r_left = np.mean(residuals[left_indices])
r_right = np.mean(residuals[right_indices])
print(f"Trung bình phần dư nhóm 1: {r_left}")
print(f"Trung bình phần dư nhóm 2: {r_right}")

lr = 0.3
leaf_1_value = lr * r_left
leaf_2_value = lr * r_right
print(f"Giá trị cập nhật nhóm 1 (lá trái): {leaf_1_value}")
print(f"Giá trị cập nhật nhóm 2 (lá phải): {leaf_2_value}")

y_pred = np.where(X <= 24, y_pred_initial + leaf_1_value, y_pred_initial + leaf_2_value)
print(f"Giá trị dự đoán mới: {y_pred}")

(b) Step1: Khởi tạo giá trị f0 dự đoán của model bằng cách lấy trung bình của Y

In [ ]:
import numpy as np

Y = np.array([50, 70, 80, 85])

f0 = np.mean(Y)
print(f"Giá trị dự đoán ban đầu f0: {f0}")

(c) Step2: Tính toán Similarity Score của root

In [ ]:
import numpy as np

Y = np.array([50, 70, 80, 85])
f0 = np.mean(Y)

lambda_ = 0

residuals = Y - f0

sum_of_residuals = np.sum(residuals)

number_of_residuals = len(Y)

similarity_score = (sum_of_residuals ** 2) / (number_of_residuals + lambda_)

print(f"Similarity Score của root: {similarity_score}")

(d) Step3: Chọn root theo từng yêu cầu sau và tính Similarity Score cho các node trong nhánh trái và nhánh phải
i. X < 23.5
ii. X < 25
iii. X < 26.5

In [ ]:
def calculate_similarity_score(residuals, lambda_):
    sum_of_residuals = np.sum(residuals)
    number_of_residuals = len(residuals)
    similarity_score = (sum_of_residuals ** 2) / (number_of_residuals + lambda_)
    return similarity_score

def split_and_calculate(X, residuals, threshold):
    left_indices = X < threshold
    right_indices = X >= threshold

    residuals_left = residuals[left_indices]
    residuals_right = residuals[right_indices]

    similarity_left = calculate_similarity_score(residuals_left, lambda_)
    similarity_right = calculate_similarity_score(residuals_right, lambda_)

    return similarity_left, similarity_right

# Điều kiện 1: X < 23.5
similarity_left_23_5, similarity_right_23_5 = split_and_calculate(X, residuals, 23.5)
print(f"X < 23.5: Similarity Score nhánh trái = {similarity_left_23_5}, nhánh phải = {similarity_right_23_5}")

# Điều kiện 2: X < 25
similarity_left_25, similarity_right_25 = split_and_calculate(X, residuals, 25)
print(f"X < 25: Similarity Score nhánh trái = {similarity_left_25}, nhánh phải = {similarity_right_25}")

# Điều kiện 3: X < 26.5
similarity_left_26_5, similarity_right_26_5 = split_and_calculate(X, residuals, 26.5)
print(f"X < 26.5: Similarity Score nhánh trái = {similarity_left_26_5}, nhánh phải = {similarity_right_26_5}")

(e) Step4: Tính Gain cho từng case i, ii, iii ở trên và chọn ra Gain có giá trị lớn nhất
• Gain = Left Similarity Score + Right Similarity Score - Root Similarity Score

In [ ]:
def calculate_similarity_score(residuals, lambda_):
    sum_of_residuals = np.sum(residuals)
    number_of_residuals = len(residuals)
    similarity_score = (sum_of_residuals ** 2) / (number_of_residuals + lambda_)
    return similarity_score

def split_and_calculate(X, residuals, threshold):
    left_indices = X < threshold
    right_indices = X >= threshold

    residuals_left = residuals[left_indices]
    residuals_right = residuals[right_indices]

    similarity_left = calculate_similarity_score(residuals_left, lambda_)
    similarity_right = calculate_similarity_score(residuals_right, lambda_)

    return similarity_left, similarity_right

root_similarity_score = calculate_similarity_score(residuals, lambda_)
print(f"Similarity Score của root: {root_similarity_score}")

similarity_left_23_5, similarity_right_23_5 = split_and_calculate(X, residuals, 23.5)
gain_23_5 = similarity_left_23_5 + similarity_right_23_5 - root_similarity_score
print(f"Gain cho X < 23.5: {gain_23_5}")

similarity_left_25, similarity_right_25 = split_and_calculate(X, residuals, 25)
gain_25 = similarity_left_25 + similarity_right_25 - root_similarity_score
print(f"Gain cho X < 25: {gain_25}")

similarity_left_26_5, similarity_right_26_5 = split_and_calculate(X, residuals, 26.5)
gain_26_5 = similarity_left_26_5 + similarity_right_26_5 - root_similarity_score
print(f"Gain cho X < 26.5: {gain_26_5}")

max_gain = max(gain_23_5, gain_25, gain_26_5)
print(f"Gain lớn nhất: {max_gain}")

(f) Step5 Sau khi chọn được model cho gain có giá trị lớn nhất ta tính giá Ouput cho từng node trong nhánh trái và phải theo công thức sau:
• Output = SumofResiduals
NumberofResiduals

• SumofResiduals là tổng của các giá trị trong Y - f0
• NumberofResiduals là số lượng sample

In [ ]:
def calculate_output(residuals):
    sum_of_residuals = np.sum(residuals)
    number_of_residuals = len(residuals)
    output = sum_of_residuals / number_of_residuals
    return output

threshold = 25
left_indices = X < threshold
right_indices = X >= threshold

residuals_left = residuals[left_indices]
residuals_right = residuals[right_indices]

output_left = calculate_output(residuals_left)
output_right = calculate_output(residuals_right)

print(f"Output cho nhánh trái: {output_left}")
print(f"Output cho nhánh phải: {output_right}")

(g) Step6: Bây giờ ta sẽ thực hiện dự đoán kết quả khi x = 25 theo công thức bên dưới
• Cần chọn nhánh phù hợp theo giá trị của X.
• Khi chọn được nhánh phù hơp ta lấy giá trị Ouput của nhánh và thực hiện dự đoán theo
công thức f0 + lr ∗ Output

In [ ]:
f0 = np.mean(Y)

lr = 0.3

threshold = 25

if 25 < threshold:
    output = output_left
else:
    output = output_right

prediction = f0 + lr * output
print(f"Dự đoán cho x = 25: {prediction}")

2. (a) Build tree với điều kiện sau λ = 0, depth = 1, lr = 0.3. Ta sẽ biến đổi giá trị của các sample
trong Y thành xác suất, tương ứng False là 0 và True là 1

In [ ]:
import numpy as np

X = np.array([23, 24, 26, 27])
Y = np.array([0, 0, 1, 1])  # False = 0, True = 1

f0 = 0
y_pred_initial = 1 / (1 + np.exp(-f0))

gradients = y_pred_initial - Y
hessians = y_pred_initial * (1 - y_pred_initial)

def calculate_output(gradients, hessians, lambda_):
    return -np.sum(gradients) / (np.sum(hessians) + lambda_)

lambda_ = 0
threshold = 25

left_indices = X < threshold
right_indices = X >= threshold

gradients_left = gradients[left_indices]
hessians_left = hessians[left_indices]
output_left = calculate_output(gradients_left, hessians_left, lambda_)

gradients_right = gradients[right_indices]
hessians_right = hessians[right_indices]
output_right = calculate_output(gradients_right, hessians_right, lambda_)

print(f"Output nhánh trái: {output_left}")
print(f"Output nhánh phải: {output_right}")

(b) Step1: Khởi tạo giá trị f0 dự đoán của model là một hằng số 0.5 (xác suất của dự đoán)

In [ ]:
import numpy as np

f0 = 0.5
y_pred_initial = 1 / (1 + np.exp(-f0))

print(f"Xác suất dự đoán ban đầu: {y_pred_initial}")

(c) Step2: Tính toán Similarity Score của root

In [ ]:
import numpy as np

Y = np.array([0, 0, 1, 1])
f0 = 0.5
previous_probability = 0.5

residuals = Y - f0
sum_of_residuals = np.sum(residuals)
hessians = previous_probability * (1 - previous_probability)
sum_of_hessians = hessians * len(Y)
lambda_ = 0

similarity_score_root = (sum_of_residuals ** 2) / (sum_of_hessians + lambda_)

print(f"Similarity Score của root: {similarity_score_root}")

(d) Step3: Chọn root theo từng yêu cầu sau và tính Similarity Score cho các node trong nhánh
trái và nhánh phải

In [ ]:
def calculate_similarity_score(residuals, hessians, lambda_):
    sum_of_residuals = np.sum(residuals)
    sum_of_hessians = hessians * len(residuals)
    similarity_score = (sum_of_residuals ** 2) / (sum_of_hessians + lambda_)
    return similarity_score

def split_and_calculate(X, residuals, threshold):
    left_indices = X < threshold
    right_indices = X >= threshold

    residuals_left = residuals[left_indices]
    residuals_right = residuals[right_indices]

    similarity_left = calculate_similarity_score(residuals_left, hessians, lambda_)
    similarity_right = calculate_similarity_score(residuals_right, hessians, lambda_)

    return similarity_left, similarity_right

similarity_left_23_5, similarity_right_23_5 = split_and_calculate(X, residuals, 23.5)
print(f"X < 23.5: Similarity Score nhánh trái = {similarity_left_23_5}, nhánh phải = {similarity_right_23_5}")

similarity_left_25, similarity_right_25 = split_and_calculate(X, residuals, 25)
print(f"X < 25: Similarity Score nhánh trái = {similarity_left_25}, nhánh phải = {similarity_right_25}")

similarity_left_26_5, similarity_right_26_5 = split_and_calculate(X, residuals, 26.5)
print(f"X < 26.5: Similarity Score nhánh trái = {similarity_left_26_5}, nhánh phải = {similarity_right_26_5}")

(e) Step4: Tính Gain cho từng case i, ii, iii ở trên và chọn ra Gain có giá trị lớn nhất

In [ ]:
root_similarity_score = calculate_similarity_score(residuals, hessians, lambda_)
print(f"Similarity Score của root: {root_similarity_score}")

similarity_left_23_5, similarity_right_23_5 = split_and_calculate(X, residuals, 23.5)
gain_23_5 = similarity_left_23_5 + similarity_right_23_5 - root_similarity_score
print(f"Gain cho X < 23.5: {gain_23_5}")

similarity_left_25, similarity_right_25 = split_and_calculate(X, residuals, 25)
gain_25 = similarity_left_25 + similarity_right_25 - root_similarity_score
print(f"Gain cho X < 25: {gain_25}")

similarity_left_26_5, similarity_right_26_5 = split_and_calculate(X, residuals, 26.5)
gain_26_5 = similarity_left_26_5 + similarity_right_26_5 - root_similarity_score
print(f"Gain cho X < 26.5: {gain_26_5}")

max_gain = max(gain_23_5, gain_25, gain_26_5)
print(f"Gain lớn nhất: {max_gain}")

(f)

In [ ]:
def calculate_output(residuals, previous_probability):
    sum_of_residuals = np.sum(residuals)
    output = sum_of_residuals / (previous_probability * (1 - previous_probability))
    return output

threshold = 25
left_indices = X < threshold
right_indices = X >= threshold

residuals_left = residuals[left_indices]
residuals_right = residuals[right_indices]

output_left = calculate_output(residuals_left, previous_probability)
print(f"Output nhánh trái: {output_left}")

output_right = calculate_output(residuals_right, previous_probability)
print(f"Output nhánh phải: {output_right}")

(g)

In [ ]:
import numpy as np

previous_probability = 0.5
lr = 0.3
def log(x):
    return np.log(x)

def predict_x_25(output, previous_probability, lr):
    log_prediction = log(previous_probability / (1 - previous_probability)) + lr * output

    probability = np.exp(log_prediction) / (1 + np.exp(log_prediction))

    return log_prediction, probability

output_right = 2.0
log_prediction, probability = predict_x_25(output_right, previous_probability, lr)

print(f"LogPrediction cho x = 25: {log_prediction}")
print(f"Xác suất dự đoán cho x = 25: {probability}")

3. (a)

In [ ]:
1 import numpy as np
2 import pandas as pd
3 import matplotlib . pyplot as plt
4 import xgboost as xgb
5 from sklearn . metrics import mean_squared_error , mean_absolute_error
6 from sklearn . preprocessing import OrdinalEncoder
7 from sklearn . model_selection import train_test_split

(b)

In [ ]:
1 dataset_path = ’/ content / Problem3 .csv ’
2 data_df = pd . read_csv ( dataset_path )
3 data_df

(c)

In [ ]:
categorical_cols = data_df . select_dtypes ( include =[ ’object ’, ’bool ’]) . columns .
to_list ()

for col_name in categorical_cols :
n_categories = data_df [ col_name ]. nunique ()
print ( f’Number of categories in { col_name }: { n_categories }’)

ordinal_encoder = OrdinalEncoder ()
encoded_categorical_cols = ordinal_encoder . fit_transform ( data_df [
categorical_cols ])

encoded_categorical_df = pd . DataFrame (
encoded_categorical_cols ,
columns = categorical_cols

numerical_df = data_df . drop ( categorical_cols , axis =1)
encoded_df = pd . concat ([ numerical_df , encoded_categorical_df ] , axis =1)

(d)

In [ ]:
 X = encoded_df . drop ( columns =[ ’area ’])
 y = encoded_df [’area ’]

(e)

In [ ]:
X_train , X_test , y_train , y_test = train_test_split (X , y ,
test_size =0.3 ,
random_state =7
)

(f)

In [ ]:
xg_reg = xgb . XGBRegressor ( seed =7 ,
learning_rate =0.01 ,
n_estimators =102 ,
max_depth =3
)

xg_reg . fit ( X_train , y_train )

(g)

In [ ]:
mae = mean_absolute_error ( y_test , preds )
mse = mean_squared_error ( y_test , preds )

print (’Evaluation results on test set:’)
print ( f’Mean Absolute Error : { mae}’)
print ( f’Mean Squared Error : { mse}’)